In [ ]:
%matplotlib qt
from bluesky import RunEngine
from bluesky.callbacks import LiveTable
from bluesky.plans import scan, count
from bluesky.plan_stubs import mv, open_run, close_run, create, \
    save, drop, rd, trigger, trigger_and_read, declare_stream
from ophyd.sim import det1, noisy_det, motor1
from databroker import Broker
from matplotlib.pyplot import ion
from bluesky.utils import install_nb_kicker
from bluesky.callbacks.best_effort import BestEffortCallback
from pprint import pprint
from IPython.display import display
install_nb_kicker()

ion()
RE = RunEngine()

db = Broker.named('temp')
RE.subscribe(db.insert)
bec = BestEffortCallback()
RE.subscribe(bec)

## Documents

Um dos principais objetivos do Bluesky é habilitar a coleta de dados e metadados. A meneira que ele faz isso é através dos **Documentos**nts.

Documentos são basicamente dicionários em Python com um esquema definido, ou seja, é um dicionário organizado de uma maneira específica e documentada. Esses **documentos** são gerados pela RunEngine durante a execução de planos e todos os dados e metadados estão organizados dentro dos **Documentos**.

### Runs e eventos

Planos podem gerar dados e metadados, porém como é possível organizar ou agrupar informações em Eventos e Runs?

In [ ]:
def one_run_one_event():
    yield from open_run()
    yield from trigger_and_read([det1, noisy_det])
    yield from close_run()

In [ ]:
RE(one_run_one_event())

Neste exemplo temos:
 - Uma tabela (uma Run)
 - Uma linha (um evento)
 - Duas colunas (além de seq_num e time), representando os dois detectores

In [ ]:
def one_run_n_events():
    yield from open_run()
    for i in range(4):
        yield from trigger_and_read([det1, noisy_det])
    yield from close_run()

In [ ]:
RE(one_run_n_events())

Neste exemplo temos:
 - Uma tabela (uma Run)
 - Quatro linhas (quatro eventos)
 - Duas colunas (além de seq_num e time), representando os dois detectores

In [ ]:
def multi_run():
    for i in range(2):
        yield from one_run_n_events()

In [ ]:
RE(multi_run())

Neste exemplo temos:
 - Duas tabelas (duas Runs)
 - Quatro linhas (quatro eventos por Run)
 - Duas colunas (além de seq_num e time), representando os dois detectores

   
Além disso, agora o retorno da função é uma tupla com multiplos uids, cada um relacionado com uma Run

A definição de uma Run é flexível, pois tudo depende de como você gostaria de organizar seus dados, seja para coleta ou análise.

#### Streams

No bluesky, as streams são fluxos de dados gerados durante uma execução de um plano. De maneira geral, cada stream contém um conjunto de dados e metadados associados a fases específicas de um experimento, por exemplo. 

Os fluxos de dados são organizados nos __event documents_ enviados pela RunEngine durante a execução de um plano

### Documentos de uma Run

#### O Start Document
O Start document contém informações de metadados no início da run:
- time - Tempo de início do plano
- plan_name - Nome do plano
- uid - ID único de identificação da Run
- scan_id - id "mais amigável", porém não é necessáriamente único
- metadados

In [ ]:
uid, = RE(scan([det1], motor1, 0, 10, 6))

In [ ]:
last_run = db[uid]

In [ ]:
last_run.start

#### O Event Document
Vão guardar informações de um ou mais eventos:

In [ ]:
for event in last_run.events():
    display(event)
    break

In [ ]:
pprint(event.to_name_dict_pair())

#### O Stop Document

In [ ]:
last_run.stop

### Metadados

Metadados são completamente personalizáveis, dado que sua classificação como dado/metadado é sensível ao contexto da aplicação.

Por exemplo, dados que precisamos saber apenas antes do experimento como:
- Nome do usuário
- Parâmaetros da amostra
- Informações da medida

Todas essas informações podem estar descritas no **Start Document**. Existem algumas maneiras de adicionar os metadados:

In [ ]:
uid, = RE(count([det1], num=10), sample_name='Agora vai', bl_operator='Alguém', minhas_informacoes='Só um teste...')

In [ ]:
last_run_with_metadata = db[uid]

In [ ]:
last_run_with_metadata.start

Planos com múltiplas Runs também vão receber os mesmos metadados:

In [ ]:
uid_run1, uid_run2, = RE(multi_run(), sample_name='Agora vai', bl_operator='Alguém', minhas_informacoes='Só um teste...')

In [ ]:
run1 = db[uid_run1]
run2 = db[uid_run2]

In [ ]:
run1.start

In [ ]:
run2.start

Também é possível definir metadados específicos para cada plano, utilizando o parâmetro **md**:

In [ ]:
def my_multi_run_plan():
    yield from scan([det1], motor1, 0, 5, 5, md={'idea': 'find the good region', 'ref': 'Ti'})
    yield from count([det1], num=10, md={'purpose': 'stab'})

In [ ]:
uid1, uid2, = RE(my_multi_run_plan(), RE_metadata={'operator': 'Alguém'}, more_meta={'meta': 'data'})

In [ ]:
multi1 = db[uid1]
multi2 = db[uid2]

In [ ]:
multi1.start

In [ ]:
multi2.start

#### Adicionando metadados para uso repetitivo

In [ ]:
RE.md

In [ ]:
RE.md['proposal'] = 20231320

In [ ]:
RE.md

In [ ]:
uid, = RE(count([det1], num=10))

In [ ]:
my_run = db[uid]

In [ ]:
my_run.start

#### Configurar validações de presença de metadados

In [ ]:
def validate_proposal_number(md):
    if 'proposal' not in md:
        raise ValueError('No proposal number was set!')

    if not isinstance(md['proposal'], int):
        raise ValueError('Proposal must be int.')

In [ ]:
RE.md_validator = validate_proposal_number

In [ ]:
del RE.md['proposal']

In [ ]:
RE(count([det1], num=3))

In [ ]:
RE(count([det1]), proposal='opa')

In [ ]:
RE.md['proposal'] = 20231320

### Supplemental Data

O Suplemental Data faz parte dos Blusky Preprocessors, basicamente preprocessors são funções que alteram planos antes de serem executados, permitindo adições de funcionalidades de maneira automática, sem que o usuário precise realizar interferências manuais em cada etapa

In [ ]:
from bluesky.preprocessors import SupplementalData
sd = SupplementalData()
RE.preprocessors.append(sd)

#### Baseline e Monitor

É um conjunto de dispositivos cujas leituras são automaticamente registradas no início e no final de cada experimento. A função baseline serve para garantir a captura de parâmetros críticos (como temperatura, pressão, etc.), permitindo comparar essas variáveis ao longo do tempo e garantir a consistência dos experimentos.

In [ ]:
sd.baseline = [det1, noisy_det, motor1]

In [ ]:
RE(count([det1], num=3))

In [ ]:
RE(multi_run())

In [ ]:
sd.monitors = [noisy_det]

In [ ]:
uid_preprocessor, = RE(count([det1], num=6))

In [ ]:
my_preprocessor_info = db[uid_preprocessor]

In [ ]:
my_preprocessor_info.stream_names

In [ ]:
my_preprocessor_info.table('baseline')

In [ ]:
my_preprocessor_info.table('noisy_det_monitor')

### Callbacks

A RunEngine, ao executar um plano, organiza dados em matadados em documentos. Cada vez que um documento é criado a RE passa essa lista de documentos para uma lista de funções.

Cada função pode ter sua implementação específica, desde salvamento de dados até plots em tempo real.

In [ ]:
bec.disable_table()
RE(count([det1], num=3), print)

In [ ]:
def my_callback(name, doc):
    print('From my callback:', name)
    print('Docs', doc)

In [ ]:
RE(count([det1], num=3), my_callback)

In [ ]:
uid, = RE(scan([det1], motor1, 0, 10, 10))

In [ ]:
my_run = db[uid]

In [ ]:
my_run.descriptors

In [ ]:
my_run.descriptors[0]['data_keys']

In [ ]:
my_run.descriptors[0]['object_keys']

In [ ]:
my_run.descriptors[0]['configuration']

In [ ]:
my_run.descriptors[0]['hints']

### Metadata on Standard Plans

In [ ]:
(uid, ) = RE(scan([det1], motor1, -3, 3, 15, md={
    "motive": "Calibration",
    "sample": "Standard Sample"
}))
db[uid]

In [ ]:
last_run = db[-1]
last_run

### Start Event

- time: Horário de início do scan

- plan_name: Nome do plano executado, como 'scan' ou 'grid_scan'

- uid: ID único que identifica o scan realizado

- scan_id: Número inteiro amigável ao usuário, mas não necessáriamente único 

- Outros metadados inseridos pelo usuário ou plano

In [ ]:
last_run.start

### Stop Event

- time: Horário de fim do scan

- exit_status: Status do plano ao final ('abort', 'fail', 'success')

In [ ]:
last_run.stop

### Descriptors

In [ ]:
last_run.descriptors[0]

### Events

In [ ]:
last_run.table()

## Metadata Plan Stubs

In [ ]:
def plan_with_data():
    yield from open_run(md={})
    
    # Plan procedures
    yield from trigger_and_read([noisy_det])
    
    yield from close_run()

uid = RE(plan_with_data(), LiveTable(['noisy_det']))
db[-1].table()

In [ ]:
db[-1]

In [ ]:
def plan_with_data():
    yield from open_run(md={})
    
    # Plan procedures
    yield from trigger_and_read([noisy_det])
    yield from rd(motor1)
    
    yield from close_run()

uid = RE(plan_with_data(), LiveTable(['noisy_det', 'motor1']))
db[-1]

In [ ]:
def plan_with_data():
    yield from open_run(md={})
    yield from create(name="custom_data_stream")
    
    # Plan procedures
    yield from trigger(noisy_det)
    yield from rd(noisy_det)
    yield from rd(motor1)

    yield from save()
    yield from close_run()

uid = RE(plan_with_data(), LiveTable(['noisy_det']))
db[-1]

In [ ]:
db[-1].table("custom_data_stream")

create: Bundle future readings into a new Event document.

save: Close a bundle of readings and emit a completed Event document.

drop: Drop a bundle of readings without emitting a completed Event document.

subscribe: Subscribe the stream of emitted documents.

unsubscribe: Remove a subscription.


Plan Stubs

trigger: Trigger and acquisition.

read: Take a reading and add it to the current bundle of readings.

rd: Reads a single-value non-triggered object

Preprocessors -> What is a decorator

relative_set_decorator: Interpret 'set' messages on devices as relative to initial position.

reset_positions_decorator: Return movable devices to their initial positions at the end.

run_decorator: Enclose in 'open_run' and 'close_run' messages.

stage_decorator: 'Stage' devices (i.e., prepare them for use, 'arm' them) and then unstage.

finalize_decorator: try...finally helper

Suplemental Data



baseline_wrapper()

monitor_during_wrapper()

fly_during_wrapper()